DATA

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
df = pd.read_csv("C:/Users/HP/Desktop/UCI_Credit_Card.csv")
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


CLEANING DATA

In [2]:
df.isnull().values.any()

False

In [3]:
df["default.payment.next.month"].value_counts()

0    23364
1     6636
Name: default.payment.next.month, dtype: int64

In [4]:
df=df.rename(columns = {'default.payment.next.month':'default'})

In [5]:
from sklearn.utils import resample
df_majority = df[df.default==0]
df_minority = df[df.default==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     
                                 n_samples=23364,    
                                 random_state=123)
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.default.value_counts()

1    23364
0    23364
Name: default, dtype: int64

In [6]:
df1 = df_upsampled
df1.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
5,6,50000.0,1,1,2,37,0,0,0,0,...,19394.0,19619.0,20024.0,2500.0,1815.0,657.0,1000.0,1000.0,800.0,0
6,7,500000.0,1,1,2,29,0,0,0,0,...,542653.0,483003.0,473944.0,55000.0,40000.0,38000.0,20239.0,13750.0,13770.0,0


In [7]:
X= df1.drop(["ID","default"],1)
y=df1["default"]

FEATURE ELIMINATION

In [8]:
import statsmodels.api as sm
cols = list(X)
p_max = 1
while (len(cols)>0):
    p=[]
    X_1= X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = list(model.pvalues.values[1:])
    p = pd.Series(p,index = cols)
    p_max = max(p)
    features_with_p_max = p.idxmax()
    if (p_max>0.05):
        cols.remove(features_with_p_max)
    else:
        break
selected_features = cols

In [9]:
cols=  selected_features
X = X[cols]

In [10]:
from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.3,random_state=42)

MODELS

In [11]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()

In [12]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [13]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
crossval_log = cross_val_score(log_reg,X_train,y_train,cv=10).mean()

In [14]:
log_reg.fit(X_train,y_train)
y_pred_log= log_reg.predict(X_test)

log_matrix = confusion_matrix(y_test, y_pred_log)
pre_log = precision_score(y_test, y_pred_log)
recall_log = recall_score(y_test, y_pred_log)

In [15]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

crossval_rf = cross_val_score(rf,X_train,y_train,cv=10).mean()

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [16]:
rf.fit(X_train,y_train)
y_pred_rf= rf.predict(X_test)

rf_matrix = confusion_matrix(y_test, y_pred_rf)
pre_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)

In [17]:
from sklearn.neighbors import KNeighborsClassifier
knn  = KNeighborsClassifier()

crossval_knn = cross_val_score(knn,X_train,y_train,cv=10).mean()

In [18]:
knn.fit(X_train,y_train)
y_pred_knn = knn.predict(X_test)

knn_matrix = confusion_matrix(y_test, y_pred_knn)
pre_knn = precision_score(y_test, y_pred_knn)
recall_knn = recall_score(y_test, y_pred_knn)

REPORT

In [19]:
Model = ["Logistic Regression","Random Forest Classifier", "K-NN Classifier"]
cross_val= [crossval_log,crossval_rf,crossval_knn]
matrix = [log_matrix,rf_matrix,knn_matrix]
precision = [pre_log,pre_rf,pre_knn]
recall = [recall_log,recall_rf,recall_knn]
results=pd.DataFrame({'Model': Model,'CrossValidation Score': cross_val,'Accuracy Matrix': matrix,'Precision': precision,'Recall': recall})
results

,Model,CrossValidation Score,Accuracy Matrix,Precision,Recall
0,Logistic Regression,0.624049,"[[3505, 3563], [1901, 5050]]",0.586323,0.726514
1,Random Forest Classifier,0.909383,"[[6339, 729], [454, 6497]]",0.899114,0.934686
2,K-NN Classifier,0.697117,"[[4412, 2656], [1450, 5501]]",0.674390,0.791397
